### Imports

In [ ]:
from sql_functions import cohen_d
from scipy.stats import ttest_ind
# from dotenv import load_dotenv if there is an Error with get_dataframe() remove this '#' and redo
from sql_functions import get_dataframe
# load_dotenv() if there is an Error with get_dataframe() remove this '#' and redo

### Excluding data

In [ ]:
# excluding test IDs, mails, free test-webinars and 'Fragestunden'(webrtc)

exclude_ids = [22223392, 21417972, 22219300, 22223207, 22223274, 19893081, 19893082,
    19347161, 19347162, 19438354, 19438355, 19438356, 19456638, 19457244,
    19457296, 19459001, 19484563, 19604300, 19604327, 19736279, 19736440,
    19736638, 19893072, 19893073, 19893074, 19893075, 19893076, 19893077,
    19893078, 19893079, 19893080, 18941278, 18974077, 19102436, 19102437,
    19102438, 19102439, 19118590, 19118591, 19118592, 19118593, 19132288,
    19132289, 19132291, 19132292, 19132849, 19132850, 19132851, 19132852,
    19134481, 19134482, 19245405, 19245406, 19245407, 19245408]

exclude_mails = ['a4d49ea13f374cd5325f430c003e755d8968be7cf8b195c597c80bf2a4529962', 
                 'd61eb5bae199f5d85f26938c4fe7cf8a6a30d0d48d35b166e621fa1abf7c2723',
                 '9f60a7ecb9b083ef5101da8f7bde1c12a5aa6ebc7042fd2071bfc067802485d3',
                 '80264d7b01caadb6df3d33ad9c7830967579f292693bbcf784d2a164af2dd316',
                 '0191ae5e56dc29349990fe692882e6254e7e632abeef55aece1e933c80e01086',
                 '9f3218274b71ec994ccf100537f7f7614e1e3c702c0c061d660d4ed9d8a7db6a']

exclude_webinars = ['Crashkurs Statistik I', 'Crashkurs Statistik II', 'Crashkurs Statistik I Kopie', 'Crashkurs Statistik II Kopie', 
                    'Testinteraktionen', 'Test on demand', 'Minutentest', 'Test2 On Demand', 'Wiederholung Köln Teil 3', 'Test demand', 
                    'Test Mail 2', 'Test für Mail', 'Fragestunde Statistik 1 Bielefeld', 'Fragestunde Düsseldorf QMI 1', 
                    'Test On Demand', 'Test 3', 'Test 1', 'Test 2 ', 'Test 2', 'Testwebinar Interaktion', 'Crashkurs Statistik 1 Duisburg-Essen 28. - 29. Januar',
                    'Crashkurs Statistik 1 Duisburg-Essen 04. - 05. Februar']

exclude_episodes = ['Crashkurs QMI Düsseldorf 14. - 15. Januar Teil 2.2', 'Crashkurs QMI Düsseldorf 19. - 20. Januar Teil 2.2', 
                    'Crashkurs QMI Düsseldorf 21. - 22. Januar Teil 2.2', 'Crashkurs Statistik 1 Bielefeld 14. - 15. Januar Teil 2.2', 
                    'Crashkurs Statistik 1 Bielefeld 19. - 20. Januar Teil 2.2', 'Crashkurs Statistik 1 Bielefeld 21. - 22. Januar Teil 2.2']

webrtc = 'webrtc'

In [ ]:
# Define and get data to clean up and test
schema_name = 'public'
table_name = 'statistics_mastertable_3'

data_to_clean = get_dataframe(f"SELECT * FROM {schema_name}.{table_name};") 
data_to_clean.info()

In [ ]:
# Removing IDs that are in exclude_ids list
clean_data = data_to_clean[~data_to_clean['id'].isin(exclude_ids)]
clean_data.shape

In [ ]:
# Removing users via encrypted email addresses from exclude_mails list
clean_data = clean_data[~clean_data['email_encrypted'].isin(exclude_mails)]
clean_data.shape

In [ ]:
# Removing webinars that were handed out for free (exclude_webinars list)
clean_data = clean_data[~clean_data['webinar_title'].isin(exclude_webinars)]
clean_data.shape

In [ ]:
clean_data = clean_data[~clean_data['episode_title'].isin(exclude_episodes)]
clean_data.shape

In [ ]:
 #Excluding webinars that were "Fragestunden" (webrtc as episode_type)
clean_data = clean_data.query("episode_type != 'webrtc'")
clean_data.shape

### Prep cleaned data for t-testing

In [ ]:
# Changing broadcast_duration and watch_duration to float64 for t-testing
clean_data = clean_data.astype({"broadcast_duration": 'float64'}) 
clean_data = clean_data.astype({"watch_duration": 'float64'}) 

#### Viewing device

In [ ]:
# Creating 2 groups for desktop and mobile
group_desktop = clean_data[clean_data['viewing_device'] == 'Desktop']
group_mobile = clean_data[clean_data['viewing_device'] == 'Mobile']

shape_desktop = group_desktop.shape
shape_mobile = group_mobile.shape

In [ ]:
# T-Test for watch duration between desktop and mobile users
print(ttest_ind(group_desktop['watch_duration'], group_mobile['watch_duration'], nan_policy='omit'))
print("Desktop:", shape_desktop, "(rows/col)")
print("Mobile:", shape_mobile, "(rows/col)")

# Calculate effect size 
print("cohen's d:", cohen_d(group_desktop["watch_duration"].dropna(),group_mobile["watch_duration"].dropna()))

#### Videocategory

In [ ]:
# Creating 2 groups for each video category
group_ondemand = clean_data[clean_data['webinar_ondemand'] == True]
group_video = clean_data[clean_data['webinar_ondemand'] == False]

shape_ondemand = group_ondemand.shape
shape_video = group_video.shape

In [ ]:
# T-Test for on-demand and video (live) watch duration

print(ttest_ind(group_ondemand['watch_duration'], group_video['watch_duration'], nan_policy='omit'))
print("Ondemand:", shape_ondemand, "(rows/col)")
print("Video:", shape_video, "(rows/col)")

# Calculate effect size
print("cohen's d:", cohen_d(group_ondemand["watch_duration"].dropna(),group_video["watch_duration"].dropna()))

In [ ]:
# T-Test for on-demand and video (live) broadcast duration
print(ttest_ind(group_ondemand['broadcast_duration'], group_video['broadcast_duration'], nan_policy='omit'))
print("Ondemand:", shape_ondemand, "(rows/col)")
print("Video:", shape_video, "(rows/col)")

# Calculate effect size
print("cohen's d:", cohen_d(group_ondemand["broadcast_duration"].dropna(),group_video["broadcast_duration"].dropna()))

In [ ]:
# T-Test for on-demand and video (live) duration-watched percentage
print(ttest_ind(group_ondemand['watch_percentage'], group_video['watch_percentage'], nan_policy='omit'))
print("Ondemand:", shape_ondemand, "(rows/col)")
print("Video:", shape_video, "(rows/col)")

# Calculate effect size
print("cohen's d:", cohen_d(group_ondemand["watch_percentage"].dropna(),group_video["watch_percentage"].dropna()))

In [ ]:
# describe the used groups
group_ondemand[['broadcast_duration','watch_duration','watch_percentage']].describe()

In [ ]:
# describe the used groups
group_video[['broadcast_duration','watch_duration','watch_percentage']].describe()